<h1>Airbnb_Data_Analytics</h1>


#### Dataset Source following link: https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata

In [ ]:
import pandas as pd


In [ ]:
## Read the csv file
data = pd.read_csv ("Airbnb_Open_Data.csv")


In [ ]:
## Display the first 5 rows
data.head(3)

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
# data of columns to drop
columns_to_keep = ['NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights','availability 365','review rate number',
       'number of reviews']

columns_to_drop = ['reviews per month', 'calculated host listings count',
       'house_rules', 'license', 'last review','id']

In [ ]:
len(columns_to_keep)

In [ ]:
len(columns_to_drop)

In [ ]:
df=data[columns_to_keep]

In [ ]:
df.shape

In [ ]:
data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
data.shape

In [ ]:
New_Columns_Names =[]
for col in data.columns:
    New_Columns_Names.append(col.upper().replace(' ', '_'))


In [ ]:
New_Columns_Names

In [ ]:
New_data = data.copy()

In [ ]:
New_data.columns = New_Columns_Names

In [ ]:
New_data.columns

In [ ]:
## Check for missing values in the dataframe and display the count in ascending order.
missing_values = New_data.isnull().sum().sort_values(ascending=True)
print("Missing Values Count:")
print(missing_values)

In [ ]:
# Check for duplicate rows and remove them
before_duplicates = len(New_data)
New_data.drop_duplicates(inplace=True)
after_duplicates = len(New_data)

# Display total number of records before and after removing duplicates
print(f"Total number of records before removing duplicates: {before_duplicates}")
print(f"Total number of records after removing duplicates: {after_duplicates}")


In [ ]:
## Rename the column.
New_data.rename(columns={'AVAILABILITY_365': 'DAYS_AVAILABLE'}, inplace=True)

In [ ]:
New_data["PRICE"][1]

In [ ]:
New_data['PRICE'] = New_data['PRICE'].str.replace('$', '')


In [ ]:
New_data["PRICE"][1]

In [ ]:
New_data['PRICE'] = New_data['PRICE'].str.replace(' ', '')


In [ ]:
New_data["PRICE"]

In [ ]:
New_data['PRICE'] = New_data['PRICE'].str.replace(',', '')

In [ ]:
type(New_data["PRICE"][1])

In [ ]:
New_data["PRICE"] = New_data["PRICE"].astype(float)

In [ ]:
New_data["SERVICE_FEE"][1]

In [ ]:
New_data['SERVICE_FEE'] = New_data['SERVICE_FEE'].str.replace('$', '')
New_data['SERVICE_FEE'] = New_data['SERVICE_FEE'].str.replace(' ', '')
New_data['SERVICE_FEE'] = New_data['SERVICE_FEE'].str.replace(',', '')
New_data["SERVICE_FEE"] = New_data["SERVICE_FEE"].astype(float)

In [ ]:
New_data["SERVICE_FEE"][1]

In [ ]:
invalid_values = New_data[(New_data['DAYS_AVAILABLE'] > 365) | (New_data['DAYS_AVAILABLE'] < 0)]
print(invalid_values['DAYS_AVAILABLE'])


In [ ]:
New_data['DAYS_AVAILABLE'] = New_data['DAYS_AVAILABLE'].fillna(0)


In [ ]:


median_value = New_data[(New_data['DAYS_AVAILABLE'] >= 0) & (New_data['DAYS_AVAILABLE'] <= 365)]['DAYS_AVAILABLE'].median()
New_data['DAYS_AVAILABLE'] = New_data['DAYS_AVAILABLE'].apply(lambda x: median_value if (x > 365 or x < 0) else x)

invalid_values = New_data[(New_data['DAYS_AVAILABLE'] > 365) | (New_data['DAYS_AVAILABLE'] < 0)]
print(invalid_values['DAYS_AVAILABLE'])


In [ ]:
Y3 =New_data["DAYS_AVAILABLE"].median()   

New_data['DAYS_AVAILABLE'] = New_data.apply(lambda row: Y3 if row['DAYS_AVAILABLE'] < row['MINIMUM_NIGHTS'] else row['DAYS_AVAILABLE'], axis=1)

print(New_data[['DAYS_AVAILABLE', 'MINIMUM_NIGHTS']].head())

In [ ]:
import numpy as np

New_data['NAME'] = New_data['NAME'].fillna('Unknown')

New_data = New_data.dropna(subset=['HOST_ID'])

New_data = New_data.dropna(subset=['HOST_IDENTITY_VERIFIED'])

New_data = New_data.dropna(subset=['HOST_NAME'])

bad_values = ["@ Art House Monique", "(Email hidden by Airbnb)", "nan", "None", ""]

New_data = New_data[~New_data['HOST_NAME'].isin(bad_values)]
New_data = New_data[~New_data['HOST_NAME'].str.isdigit()]

# NEIGHBOURHOOD_GROUP
New_data = New_data.dropna(subset=['NEIGHBOURHOOD_GROUP'])

# NEIGHBOURHOOD
New_data = New_data.dropna(subset=['NEIGHBOURHOOD'])

# ROOM_TYPE
New_data = New_data.dropna(subset=['ROOM_TYPE'])

# LAT & LONG → If missing, drop the row since they are essential for location
New_data = New_data.dropna(subset=['LAT', 'LONG'])

# COUNTRY & COUNTRY_CODE
New_data['COUNTRY'] = New_data['COUNTRY'].fillna('United States')
New_data['COUNTRY_CODE'] = New_data['COUNTRY_CODE'].fillna('US')

# INSTANT_BOOKABLE
New_data['INSTANT_BOOKABLE'] = New_data['INSTANT_BOOKABLE'].fillna('Unknown')

# CANCELLATION_POLICY
New_data['CANCELLATION_POLICY'] = New_data['CANCELLATION_POLICY'].fillna('Unknown')

# CONSTRUCTION_YEAR → If missing, fill with median; if less than 1800, set to NaN
New_data.loc[New_data['CONSTRUCTION_YEAR'] < 1800, 'CONSTRUCTION_YEAR'] = np.nan
New_data['CONSTRUCTION_YEAR'] = New_data['CONSTRUCTION_YEAR'].fillna(
    New_data['CONSTRUCTION_YEAR'].median()
)

# PRICE → Remove values <= 0, fill remaining missing values with median
New_data.loc[New_data['PRICE'] <= 0, 'PRICE'] = np.nan
New_data['PRICE'] = New_data['PRICE'].fillna(New_data['PRICE'].median())

# SERVICE_FEE → Same logic as price
New_data.loc[New_data['SERVICE_FEE'] < 0, 'SERVICE_FEE'] = np.nan
New_data['SERVICE_FEE'] = New_data['SERVICE_FEE'].fillna(
    New_data['SERVICE_FEE'].median()
)

# MINIMUM_NIGHTS → Very large values (> 365) are considered outliers and replaced with median
New_data.loc[New_data['MINIMUM_NIGHTS'] > 365, 'MINIMUM_NIGHTS'] = np.nan
New_data['MINIMUM_NIGHTS'] = New_data['MINIMUM_NIGHTS'].fillna(
    New_data['MINIMUM_NIGHTS'].median()
)
New_data = New_data[New_data['MINIMUM_NIGHTS'] >= 1]

# NUMBER_OF_REVIEWS → Fill missing values with 0
New_data['NUMBER_OF_REVIEWS'] = New_data['NUMBER_OF_REVIEWS'].fillna(0)

# REVIEW_RATE_NUMBER → Fill missing values with median
New_data['REVIEW_RATE_NUMBER'] = New_data['REVIEW_RATE_NUMBER'].fillna(
    New_data['REVIEW_RATE_NUMBER'].median()
)


In [ ]:
# corrections = {
#     'brookln': 'Brooklyn',
#     'manhatan': 'Manhattan'
# }
# New_data['NEIGHBOURHOOD_GROUP'] = New_data['NEIGHBOURHOOD_GROUP'].replace(corrections)
# New_data['NEIGHBOURHOOD_GROUP'].unique()


In [ ]:
New_data.dtypes

In [ ]:
New_data['HOST_ID'] = New_data['HOST_ID'].astype('int64')
New_data['CONSTRUCTION_YEAR'] = New_data['CONSTRUCTION_YEAR'].astype('int64')
New_data['MINIMUM_NIGHTS'] = New_data['MINIMUM_NIGHTS'].astype('int64')
New_data['NUMBER_OF_REVIEWS'] = New_data['NUMBER_OF_REVIEWS'].astype('int64')
New_data['DAYS_AVAILABLE'] = New_data['DAYS_AVAILABLE'].astype('int64')

In [ ]:
New_data.columns

In [ ]:

numeric_columns = [ 'CONSTRUCTION_YEAR', 'PRICE', 'SERVICE_FEE', 
                    'NUMBER_OF_REVIEWS', 'REVIEW_RATE_NUMBER', 'DAYS_AVAILABLE']

def find_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers

for column in numeric_columns:
    outliers = find_outliers(New_data, column)
    print(f"\nالعمود: {column}")
    print(f"عدد القيم المتطرفة: {len(outliers)}")
    if len(outliers) > 0:
        print(f"أمثلة على القيم المتطرفة: {outliers.head().to_list()}")
    else:
        print("لا توجد قيم متطرفة.")

print("\nإحصائيات الأعمدة العددية:")
print(New_data[numeric_columns].describe())

In [ ]:
def replace_outliers(New_data, column):
    Q1 = New_data[column].quantile(0.25)
    Q3 = New_data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    median = New_data[column].median()
    New_data[column] = np.where((New_data[column] < lower_bound) | (New_data[column] > upper_bound), median, New_data[column])
    return New_data


In [ ]:

# استبدال القيم المتطرفة بالوسيط لكل عمود عددي
for column in numeric_columns:
    New_data = replace_outliers(New_data, column)
    print(f"\nتم استبدال القيم المتطرفة في العمود {column} بالوسيط.")


In [ ]:
# عرض القيم الفريدة لكل عمود
print("Unique values in NEIGHBOURHOOD_GROUP:")
print(New_data['NEIGHBOURHOOD_GROUP'].unique())
print("-------------------------------------------------")
print("Unique values in ROOM_TYPE:")
print(New_data['ROOM_TYPE'].unique())

In [ ]:
New_data.to_csv('data_cleaned_replaceNullv4.csv')
New_data.to_excel('data_cleaned_replaceNullv4.xlsx')


# صفحة 1: Listings & Hosts


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# نفترض إن الداتا عندك متخزنة في DataFrame باسم New_data
# لو لسه CSV عندك: استخدم السطر ده
# New_data = pd.read_csv("airbnb.csv")

# -------------------------------
# صفحة 1: Listings & Hosts
# -------------------------------

# Bar Chart: عدد العقارات حسب Room Type
New_data['ROOM_TYPE'].value_counts().plot(kind='bar')
plt.title("Number of Listings by Room Type")
plt.xlabel("Room Type")
plt.ylabel("Count")
plt.show()

In [ ]:
# Pie Chart: نسبة المضيفين Verified vs Not Verified
New_data['HOST_IDENTITY_VERIFIED'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title("Host Identity Verified")
plt.ylabel("")
plt.show()

In [ ]:
# Column Chart: عدد العقارات لكل Neighbourhood
New_data['NEIGHBOURHOOD'].value_counts().head(10).plot(kind='bar')
plt.title("Top 10 Neighbourhoods by Listings")
plt.xlabel("Neighbourhood")
plt.ylabel("Count")
plt.show()

# صفحة 2: Pricing


In [ ]:
# Column Chart: متوسط السعر لكل Neighbourhood (Top 10)
New_data.groupby('NEIGHBOURHOOD')['PRICE'].mean().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title("Average Price by Top 10 Neighbourhoods")
plt.xlabel("Neighbourhood")
plt.ylabel("Avg Price")
plt.show()

# صفحة 3: Reviews


In [ ]:
# 2- Barplot: متوسط التقييم لكل Neighbourhood Group
plt.figure(figsize=(7,5))
New_data.groupby("NEIGHBOURHOOD_GROUP")['REVIEW_RATE_NUMBER'].mean().plot(kind='bar')
plt.title("Average Review Rating by Neighbourhood Group")
plt.xlabel("Neighbourhood Group")
plt.ylabel("Average Rating")
plt.show()

In [ ]:
# Pie Chart: توزيع العقارات حسب مستويات التقييم
New_data['REVIEW_RATE_NUMBER'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title("Distribution by Review Rating")
plt.ylabel("")
plt.show()

# صفحة 4: Availability


In [ ]:
# Bar Chart: متوسط Days Available حسب Room Type
New_data.groupby('ROOM_TYPE')['DAYS_AVAILABLE'].mean().plot(kind='bar')
plt.title("Average Days Available by Room Type")
plt.xlabel("Room Type")
plt.ylabel("Avg Days Available")
plt.show()

# صفحة 5: Policies


In [ ]:
New_data['CANCELLATION_POLICY'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title("Cancellation Policy Distribution")
plt.ylabel("")
plt.show()

More ideas

2️⃣ Stacked Column: متوسط التقييم لكل Room Type

In [ ]:
review_room = New_data.groupby('ROOM_TYPE')['REVIEW_RATE_NUMBER'].mean()

review_room.plot(kind='bar', figsize=(6,5))
plt.title("Average Review Rating per Room Type")
plt.xlabel("Room Type")
plt.ylabel("Average Rating")
plt.show()


7️⃣ Line Chart: عدد العقارات مقابل سنة الإنشاء

In [ ]:
year_count = New_data.groupby('CONSTRUCTION_YEAR').size()

year_count.plot(kind='line', marker='o', figsize=(8,5))
plt.title("Number of Properties per Construction Year")
plt.xlabel("Construction Year")
plt.ylabel("Count")
plt.show()


8️⃣ Histogram: توزيع Minimum Nights

In [ ]:
plt.figure(figsize=(7,5))
plt.hist(New_data['MINIMUM_NIGHTS'], bins=50)
plt.title("Distribution of Minimum Nights")
plt.xlabel("Minimum Nights")
plt.ylabel("Frequency")
plt.xlim(0, 100)  # لتوضيح الشكل (ممكن فيه Outliers)
plt.show()


9️⃣ Column Chart: متوسط DAYS_AVAILABLE لكل Neighbourhood Group

In [ ]:
days_group = New_data.groupby('NEIGHBOURHOOD_GROUP')['DAYS_AVAILABLE'].mean()

days_group.plot(kind='bar', figsize=(7,5))
plt.title("Average Days Available per Neighbourhood Group")
plt.xlabel("Neighbourhood Group")
plt.ylabel("Average Days Available")
plt.show()
